# Autoencoder

In [1]:
import os

from utils.loaders import load_mnist
from models.AE import Autoencoder

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.75
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

## Set parameters

In [2]:
# run params
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

## Load the data

In [3]:
(x_train, y_train), (x_test, y_test) = load_mnist()

## Define the structure of the neural network

In [4]:
AE = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if MODE == 'build':
    AE.save(RUN_FOLDER)
else:
    AE.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [5]:
AE.encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder_conv_0 (Conv2D)      (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 32)        0         
_________________________________________________________________
encoder_conv_1 (Conv2D)      (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
encoder_conv_2 (Conv2D)      (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 64)          0     

In [6]:
AE.decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0   

## Train the autoencoder

In [7]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0

In [8]:
AE.compile(LEARNING_RATE)

In [9]:
AE.train(     
    x_train[:1000]
    , batch_size = BATCH_SIZE
    , epochs = 200
    , run_folder = RUN_FOLDER
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
32/32 [==============================] - ETA: 0s - loss: 0.1821
Epoch 00001: saving model to run/vae/0001_digits/weights/weights.h5
32/32 [==============================] - 0s 14ms/step - loss: 0.1821 - lr: 5.0000e-04
Epoch 2/200
28/32 [=========================>....] - ETA: 0s - loss: 0.0791
Epoch 00002: saving model to run/vae/0001_digits/weights/weights.h5
32/32 [==============================] - 0s 11ms/step - loss: 0.0780 - lr: 5.0000e-04
Epoch 3/200
28/32 [=========================>....] - ETA: 0s - loss: 0.0669
Epoch 00003: saving model to run/vae/0001_digits/weights/weights.h5
32/32 [==============================] - 0s 11ms/step - loss: 0.0668 - lr: 5.0000e-04
Epoch 4/200
28/32 [=========================>....] - ETA: 0s - loss: 0.0622
Epoch 00004: saving model to run/vae/0001_digits/weights/weights.h5
32/32 [==============================] - 0s 11ms/step - loss: 0.0621 - lr: 5.0000e-04
Epoch 5/200
28/32 [=========================>....] - ETA: 0s - loss: 0.0594
Epoc

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

print(tf.__version__)
print(tf.test.is_gpu_available())
